In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

In [2]:
data = pd.read_csv("C:\\Users\\racha\\OneDrive\\Desktop\\ddos_data.csv")

In [3]:
data['Time'] = data['Time'].apply(lambda x: x.split(':'))
data['hour'] = data['Time'].apply(lambda x: int(x[0]))
data['minute_int'] = data['Time'].apply(lambda x: int(x[1].split('.')[0]))
data['minute_dec'] = data['Time'].apply(lambda x: int(x[1].split('.')[1]))


In [4]:
data['second'] = data['Time'].apply(lambda x: 
                                     float(x[2]) if len(x) >= 3 else 0.0)

In [5]:
data.drop('Time', axis=1, inplace=True)

In [6]:
X = data[['hour', 'minute_int', 'minute_dec', 'second', 'Source', 'Destination', 'Protocol', 'Length', 'Src port', 'Dest port']]
y = data['Target']

In [7]:
X_encoded = pd.get_dummies(X, columns=['Source', 'Destination','Protocol'])

In [8]:
X_train_encoded, X_test_encoded, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [9]:
model = RandomForestClassifier()
model.fit(X_train_encoded, y_train)

RandomForestClassifier()

In [10]:
y_pred = model.predict(X_test_encoded)

In [11]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


# AS code


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [25]:
data = pd.read_csv("C:\\Users\\racha\\OneDrive\\Desktop\\ddos_data.csv") 

In [33]:
X = data.drop(['Target', 'Time', 'Source', 'Destination', 'Protocol'], axis=1)
X = X.select_dtypes(include=['float64', 'int64'])
y = data['Target']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [36]:
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

LogisticRegression()

In [37]:
predictions = model.predict(X_test_scaled)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.78      0.88      4167
           1       0.91      1.00      0.95      8749

    accuracy                           0.93     12916
   macro avg       0.95      0.89      0.92     12916
weighted avg       0.94      0.93      0.93     12916



In [38]:
from sklearn.metrics import accuracy_score

# Calculate accuracy on the test set
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.9306286776091669


# live n/w capture

In [1]:
pip install pyshark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install asyncio

Defaulting to user installation because normal site-packages is not writeable
                                              0.0/101.8 kB ? eta -:--:--
     -------------------------------         81.9/101.8 kB 2.3 MB/s eta 0:00:01
     ------------------------------------ 101.8/101.8 kB 973.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import asyncio
import pyshark

# Create a new event loop
loop = asyncio.new_event_loop()

# Set the new event loop for pyshark
pyshark.config.async_capable = True
pyshark.config.use_after_fork = True
pyshark.config.loop = loop

# Use Pyshark as usual
cap = pyshark.LiveCapture(interface='Wi-Fi')
for packet in cap.sniff_continuously():
    print(packet)


RuntimeError: Cannot run the event loop while another loop is running

In [6]:
import asyncio

def check_running_loop():
    running_loop = asyncio.get_running_loop()
    if running_loop is None:
        print("No event loop is running.")
    else:
        print(f"Event loop {running_loop} is running.")
    return running_loop

check_running_loop()


Event loop <_WindowsSelectorEventLoop running=True closed=False debug=False> is running.


<_WindowsSelectorEventLoop running=True closed=False debug=False>

In [3]:
import asyncio

async def main():
    print("Event loop running...")
    await asyncio.sleep(5)
    print("Event loop finished.")

# Create a new event loop
loop = asyncio.new_event_loop()

try:
    # Set the new event loop
    asyncio.set_event_loop(loop)
    # Run the event loop
    loop.run_until_complete(main())
finally:
    # Close the event loop
    loop.close()
    print("Event loop closed.")


Event loop closed.


RuntimeError: Cannot run the event loop while another loop is running

In [4]:
import scapy

ModuleNotFoundError: No module named 'scapy'

In [5]:
pip install scapy

Defaulting to user installation because normal site-packages is not writeable
  Using cached scapy-2.5.0-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import scapy


In [2]:
from scapy.all import sniff, IP

# Function to extract features from packets
def extract_features(packet):
    if IP in packet:
        src_ip = packet[IP].src
        dst_ip = packet[IP].dst
        packet_len = len(packet)
        return src_ip, dst_ip, packet_len
    return None

# Callback function to process each packet
def process_packet(packet):
    features = extract_features(packet)
    if features:
        src_ip, dst_ip, packet_len = features
        print(f"Source IP: {src_ip}, Destination IP: {dst_ip}, Packet Length: {packet_len}")

# Capture live network traffic
sniff(prn=process_packet, count=10)  # Capture 10 packets


Source IP: 172.25.5.215, Destination IP: 172.25.5.73, Packet Length: 164
Source IP: 172.25.5.73, Destination IP: 172.25.5.215, Packet Length: 164
Source IP: 172.25.5.215, Destination IP: 172.25.5.73, Packet Length: 54
Source IP: 172.25.0.175, Destination IP: 224.0.0.251, Packet Length: 85
Source IP: 172.25.3.161, Destination IP: 239.255.255.250, Packet Length: 217


<Sniffed: TCP:3 UDP:3 ICMP:0 Other:4>

In [15]:
from scapy.all import sniff
from scapy.layers.inet import IP, TCP, UDP
import time

def extract_packet_info(packet):
    if IP in packet:
        src_ip = packet[IP].src
        dst_ip = packet[IP].dst
        protocol = packet[IP].proto
        if 'TCP' in packet:
            protocol = 'TCP'
            src_port = packet['TCP'].sport
            dst_port = packet['TCP'].dport
        elif 'UDP' in packet:
            protocol = 'UDP'
            src_port = packet['UDP'].sport
            dst_port = packet['UDP'].dport
        else:
            protocol = 'unknown'
            src_port = 'unknown'
            dst_port = 'unknown'
            
        packet_len = len(packet)
        timestamp = datetime.now().strftime("%H:%M.%S")
        return src_ip, dst_ip, src_port, dst_port, packet_len, protocol,timestamp
    return None, None, None, None, None, None, None

def packet_callback(packet):
    src_ip, dst_ip, src_port, dst_port, packet_len, protocol,timestamp = extract_packet_info(packet)
    if src_ip:
        #print(f"Source IP: {src_ip}, Destination IP: {dst_ip}, Source Port: {src_port}, Destination Port: {dst_port}, Packet Length: {packet_len}, Protocol: {protocol}")
        with open('captured_traffic.csv', 'a') as file:
            file.write(f"{src_ip},{dst_ip},{src_port},{dst_port},{packet_len},{protocol},{timestamp}\n")

# Write the headings to the CSV file
with open('captured_traffic.csv', 'w') as file:
    file.write("Source IP,Destination IP,Source Port,Destination Port,Packet Length,Protocol,Time\n")

# Capture live network traffic for one minute
start_time = time.time()
while time.time() - start_time < 60:
    sniff(prn=packet_callback, count=1)
